# Dataset generation

(by: Alejandra Rubio Muñoz)

To use the Google colab version of the CARE (3D) tool we need to provide an image dataset to train the model. To do this, we need pairs of low-high resolution images in .tiff format to train the model, and once we have a trained model we will use it to predict and generate new high resolution images from the low resolution images we provide.

In [16]:
# imports

from readlif.reader import LifFile
import numpy as np
import tifffile as tf
import os
import SimpleITK as sitk
from aicsimageio import AICSImage
import shutil
from skimage import io, img_as_uint
from skimage.util import random_noise
import xml.etree.ElementTree as ET

In [17]:
# set base path
base_path = '/Users/alejandrarubiomunoz/Downloads/res'

## 1. Coversion of image formats to .tiff

If the format of the original images is .lif use this first function:

In [18]:
def lif_to_tiff(base_path, input_file, dimensions=None):
    '''extract imgs with specified dimensions or extract all imgs from .lif file and convert to .tiff imgs'''    
    
    # paths
    lif_path = os.path.join(base_path, input_file)
    lif_file = LifFile(lif_path) # LifFile object using the lif_path

    folder_name = 'train_up'
    output_folder = os.path.join(base_path, folder_name)
    if folder_name not in os.listdir(base_path):
        os.makedirs(output_folder)

    # iterating through each image in the .lif file
    for idx, image in enumerate(lif_file.get_iter_image()):
        # check if dimensions are not specified or the img dimensions match the specified dimensions
        if dimensions is None or (image.dims.x, image.dims.y) == dimensions:
            num_slices = image.info['dims'].z # num of slices in the img
            num_channels = image.info['channels'] #num of channels in the img

            # create an empty array to stack imgs (considering variable channels)
            image_stack = np.empty((num_channels * num_slices, image.dims.y, image.dims.x), dtype=np.uint16)

            # loop through slices and channels to populate the img stack
            for z in range(num_slices):
                for c in range(num_channels):
                    frame = np.array(image.get_frame(z=z, c=c)) # get the frame at slice z and channel c
                    image_stack[z * num_channels + c] = frame # assign the frame to the appropriate position in the stack

            voxel_size = image.info['scale'][2::-1] # voxel size info
            resolution = (voxel_size[1], voxel_size[2]) # define resolution based on voxel size

            # output file path for the .tiff img
            output_file_path = os.path.join(output_folder, f'image_{idx + 1}.tiff')
            
            # save the img stack as a .tiff file with specified metadata
            tf.imwrite(output_file_path, image_stack, resolution=resolution,
                       metadata={'axes': 'ZYX', 'Z': num_channels*num_slices})
                       
    return output_folder # get the path of the output_folder to be used in the subsequent step 2 as input folder

In [19]:
# .lif file name
input_file_lif = 'ResonantRestoration.lif'
# specify the dimensions if needed (x and y dimensions)
specified_dimensions = (2048, 2048)
# call function with specified dimensions or 'None' for extracting all images
output_folder = lif_to_tiff(base_path, input_file_lif, dimensions=specified_dimensions)


If the format of the original images is .czi use this second function:

In [13]:
def czi_to_tiff(base_path, input_file):
    '''convert .czi img to .tiff img'''

    # paths
    czi_path = os.path.join(base_path, input_file)

    folder_name = 'train_up'
    output_folder = os.path.join(base_path, folder_name)
    if folder_name not in os.listdir(base_path):
        os.makedirs(output_folder)

    # read the .czi imge using AICSImage
    img = AICSImage(czi_path)
    data = img.get_image_data('CZYX')  # img data

    # num of channels and slices from the img, and y and x dimenisons
    num_channels = data.shape[0]
    num_slices = data.shape[1]
    dim_y = data.shape[2]
    dim_x = data.shape[3]

    # empty array to combine data from all channels and slices
    image_stack = np.empty((num_channels * num_slices, dim_y, dim_x), dtype=np.uint16)

    # loop through slices and channels to populate the img stack
    for z in range(num_slices):
        for c in range(num_channels):
            frame = np.array(data[c, z, :, :])  # get the frame at slice z and channel c
            image_stack[z * num_channels + c] = frame  # assign the frame to the appropriate position in the stack

    # voxel size info
    # assuming img.metadata is an Element object
    metadata_xml = img.metadata

    # helper function to safely extract text content from an element and convert to float
    def get_element_float(element, tag):
        child = element.find(tag)
        return float(child.text) if child is not None and child.text is not None else None

    # access common metadata keys from the XML structure
    voxel_size_x = (get_element_float(metadata_xml, './/ScalingX'))* 1_000_000
    voxel_size_y = (get_element_float(metadata_xml, './/ScalingY'))* 1_000_000
    resolution = (voxel_size_x, voxel_size_y)  # define resolution (x, y)

    # save the img stack as .tiff with associated metadata
    output_file_path = os.path.join(output_folder, f'image.tiff')
    tf.imwrite(output_file_path, image_stack, resolution=resolution,
                metadata={'axes': 'ZYX', 'Z': num_channels*num_slices})

    return output_folder  # get the path of the output_folder to be used in the subsequent step 2 as input folder

In [14]:
# .czi file name
input_file_czi = 'data8channels-003.czi'
# call function
output_folder = czi_to_tiff(base_path, input_file_czi)


## 2. Virtual generation of the lower resolution image pairs

To use the CARE (3D) tool, we will need to have pairs of images (low-high resolution) to train the model. If we only have high-resolution images, we will have to virtually generate the corresponding low-resolution images, but the tool in its Google colab version only supports pairs of images of the same dimensions so we have to keep these during the procedure.\
To simulate a possible low quality real image that we will have to restore, here we scale down our original images, then apply noise and rescale  again, achieving the loss of quality we need thanks to the method used.

### A. Down and upscale original images

In [10]:
def tiff_imgs_pairs(base_path, original_images_folder, down_factor_xy=None, delete_intermediate=False):
    ''' downscale and upscale pairs of .tiff imgs in separate folders '''

    # Downscale

    #paths
    down_folder_name = 'down'
    output_folder_down = os.path.join(base_path, down_folder_name)
    if down_folder_name not in os.listdir(base_path):
        os.makedirs(output_folder_down)

    file_list = os.listdir(original_images_folder)

    original_size_dict = {}

    for file_name in file_list:
        if file_name.endswith('.tiff'):
            input_image_path = os.path.join(original_images_folder, file_name)
            output_image_path = os.path.join(output_folder_down, file_name)
            
            # read img
            img = sitk.ReadImage(input_image_path)

            # original dimensions [X, Y, Z]
            original_size = img.GetSize()
            original_size_dict[file_name] = original_size

            # calculate new dimensions [X, Y]
            new_size = [int(sz / down_factor_xy) if idx != 2 else sz for idx, sz in enumerate(original_size)]

            # calculate scale factor for new spacing
            scale_factor = [old_size / new_size_val for old_size, new_size_val in zip(original_size, new_size)]

            # calculate new spacing
            new_spacing = [sz * scale_factor_val for sz, scale_factor_val in zip(img.GetSpacing(), scale_factor)]

            # apply resolution reduction
            resampler = sitk.ResampleImageFilter()
            resampler.SetOutputSpacing(new_spacing)
            resampler.SetSize(new_size)
            resampler.SetOutputDirection(img.GetDirection())
            resampler.SetOutputOrigin(img.GetOrigin())
            resampler.SetTransform(sitk.Transform())
            resampler.SetDefaultPixelValue(img.GetPixelIDValue())

            # execute resampling
            output_image = resampler.Execute(img)

            # save images
            sitk.WriteImage(output_image, output_image_path)

    # Upscale

    # paths
    up_folder_name = 'up'
    output_folder_up = os.path.join(base_path, up_folder_name)
    if up_folder_name not in os.listdir(base_path):
        os.makedirs(output_folder_up)

    file_list = os.listdir(output_folder_down)
    
    for file_name in file_list:
        if file_name.endswith('.tiff'):
            input_image_path = os.path.join(output_folder_down, file_name)
            output_image_path = os.path.join(output_folder_up, file_name)
            
            # read img
            img = sitk.ReadImage(input_image_path)

            # original size based on filename
            original_size = original_size_dict.get(file_name)
            if original_size is None:
                continue  # skip imgs without original size info

            # new dimensions to the original size
            new_size = original_size

            # calculate scale factor for new spacing
            scale_factor = [old_size / new_size_val for old_size, new_size_val in zip(img.GetSize(), new_size)]

            # calculate new spacing
            new_spacing = [sz * scale_factor_val for sz, scale_factor_val in zip(img.GetSpacing(), scale_factor)]
            
            # apply resolution increase
            resampler = sitk.ResampleImageFilter()
            resampler.SetOutputSpacing(new_spacing)
            resampler.SetSize(new_size)
            resampler.SetOutputDirection(img.GetDirection())
            resampler.SetOutputOrigin(img.GetOrigin())
            resampler.SetTransform(sitk.Transform())
            resampler.SetDefaultPixelValue(img.GetPixelIDValue())

            # execute resampling
            output_image = resampler.Execute(img)

            # save images
            sitk.WriteImage(output_image, output_image_path)

    # delete intermediate folder
    if delete_intermediate:
        shutil.rmtree(output_folder_down)

    return output_folder_up

In [ ]:
down_factor_xy = 10  # adjust as needed
delete_intermediate = True  # set to True if you want to delete the intermediate folder
# call function
output_folder_up = tiff_imgs_pairs(base_path, output_folder, down_factor_xy, delete_intermediate)

### B. Add noise to the resutling re-upscaled images from previous step

In [148]:
def apply_gaussian_noise_to_images(base_path, output_folder_up, delete_output_folder_up=False, var_range=(0, 0.001)):
    ''' read each img from the 'output_folder_up', applies Gaussian noise to it
    with a randomly selected variance within the specified range, converts the noisy image
    to a 16-bit unsigned integer format, and saves the resulting noisy .tiff imgs
    with LZW compression within the 'train_down' folder '''

    # paths
    folder_name = 'train_down'
    output_folder = os.path.join(base_path, folder_name)
    if folder_name not in os.listdir(base_path):
        os.makedirs(output_folder)

    # file names in the provided 'output_folder_up' directory
    file_list = os.listdir(output_folder_up)
    
    for file_name in file_list:
        # read the source img from the 'output_folder_up' directory
        img = io.imread(os.path.join(output_folder_up, file_name))

        # apply Gaussian noise to the source img
        noise_var = np.random.uniform(var_range[0], var_range[1])
        noisy_img = random_noise(img, mode='gaussian', mean=0.0, var=noise_var, clip=True)

        # convert imgs to uint16 format (16 bits per pixel)
        noisy_img_16bit = img_as_uint(noisy_img)

        # save the .tiff img with LZW compression
        io.imsave(os.path.join(output_folder, file_name), noisy_img_16bit, compression='lzw')
    
    # delete the 'output_folder_up' directory and its contents
    if delete_output_folder_up:
        shutil.rmtree(output_folder_up)

In [149]:
# call function
delete_output_folder_up = True  # set to True if you want to delete the 'output_folder_up' directory and its contents
apply_gaussian_noise_to_images(base_path, output_folder_up, delete_output_folder_up)

In [43]:
def tiff_imgs_pairs(base_path, original_images_folder, down_factor_xy=None, delete_intermediate=False, var_range=(0, 0.003)):
    ''' downscale and upscale pairs of .tiff imgs in separate folders '''

    # Downscale

    # paths
    down_folder_name = 'down'
    output_folder_down = os.path.join(base_path, down_folder_name)
    if down_folder_name not in os.listdir(base_path):
        os.makedirs(output_folder_down)

    file_list = os.listdir(original_images_folder)

    original_size_dict = {}

    for file_name in file_list:
        if file_name.endswith('.tiff'):
            input_image_path = os.path.join(original_images_folder, file_name)
            output_image_path = os.path.join(output_folder_down, file_name)
            
            # Read image
            img = sitk.ReadImage(input_image_path)

            # Original dimensions [X, Y, Z]
            original_size = img.GetSize()
            original_size_dict[file_name] = original_size

            # Calculate new dimensions [X, Y]
            new_size = [int(sz / down_factor_xy) if idx != 2 else sz for idx, sz in enumerate(original_size)]

            # Calculate scale factor for new spacing
            scale_factor = [old_size / new_size_val for old_size, new_size_val in zip(original_size, new_size)]

            # Calculate new spacing
            new_spacing = [sz * scale_factor_val for sz, scale_factor_val in zip(img.GetSpacing(), scale_factor)]

            # Apply resolution reduction
            resampler = sitk.ResampleImageFilter()
            resampler.SetOutputSpacing(new_spacing)
            resampler.SetSize(new_size)
            resampler.SetOutputDirection(img.GetDirection())
            resampler.SetOutputOrigin(img.GetOrigin())
            resampler.SetTransform(sitk.Transform())
            resampler.SetDefaultPixelValue(img.GetPixelIDValue())

            # Execute resampling
            output_image = resampler.Execute(img)

            # Save images
            sitk.WriteImage(output_image, output_image_path)

    # Gaussian noise
    
    # Paths
    noise_folder_name = 'noise'
    output_folder_noise = os.path.join(base_path, noise_folder_name)
    if noise_folder_name not in os.listdir(base_path):
        os.makedirs(output_folder_noise)

    file_list = os.listdir(output_folder_down)
    
    for file_name in file_list:
        # Read the source image from the 'output_folder_down' directory
        input_image_path = os.path.join(output_folder_down, file_name)
        output_image_path = os.path.join(output_folder_noise, file_name)
        
        img = io.imread(input_image_path)

        # Apply Gaussian noise to the source image
        noise_var = np.random.uniform(var_range[0], var_range[1])
        noisy_img = random_noise(img, mode='gaussian', mean=0.0, var=noise_var, clip=True)

        # Convert images to uint16 format (16 bits per pixel)
        noisy_img_16bit = img_as_uint(noisy_img)

        # Save the .tiff image with LZW compression
        io.imsave(output_image_path, noisy_img_16bit, compression='lzw')
    
      
    # Upscale

    # Paths
    up_folder_name = 'train_down'
    output_folder_up = os.path.join(base_path, up_folder_name)
    if up_folder_name not in os.listdir(base_path):
        os.makedirs(output_folder_up)

    file_list = os.listdir(output_folder_noise)
    
    for file_name in file_list:
        if file_name.endswith('.tiff'):
            input_image_path = os.path.join(output_folder_noise, file_name)
            output_image_path = os.path.join(output_folder_up, file_name)
            
            # Read image
            img = sitk.ReadImage(input_image_path)

            # Original size based on filename
            original_size = original_size_dict.get(file_name)
            if original_size is None:
                continue  # Skip images without original size info

            # New dimensions to the original size
            new_size = original_size

            # Calculate scale factor for new spacing
            scale_factor = [old_size / new_size_val for old_size, new_size_val in zip(img.GetSize(), new_size)]

            # Calculate new spacing
            new_spacing = [sz * scale_factor_val for sz, scale_factor_val in zip(img.GetSpacing(), scale_factor)]
            
            # Apply resolution increase
            resampler = sitk.ResampleImageFilter()
            resampler.SetOutputSpacing(new_spacing)
            resampler.SetSize(new_size)
            resampler.SetOutputDirection(img.GetDirection())
            resampler.SetOutputOrigin(img.GetOrigin())
            resampler.SetTransform(sitk.Transform())
            resampler.SetDefaultPixelValue(img.GetPixelIDValue())

            # Execute resampling
            output_image = resampler.Execute(img)

            # Save images
            sitk.WriteImage(output_image, output_image_path)

    # Delete intermediate folders
    if delete_intermediate:
        shutil.rmtree(output_folder_down)
        shutil.rmtree(output_folder_noise)


In [44]:
down_factor_xy = 2  # Adjust as needed
#delete_intermediate = True  # Set to True if you want to delete the intermediate folders

# Call function
tiff_imgs_pairs(base_path, output_folder, down_factor_xy, delete_intermediate=True)